# Quantum Circuit Simulator

In [1]:
import numpy as np
from numpy.random import choice

## Define unitary matrices

#### Unitary gates

In [2]:
isqrt2 = 1/np.sqrt(2) # inverse of the square root of 2

x = np.array([
    [0, 1],
    [1, 0]
])

z = np.array([
    [1, 0],
    [0, -1]
])

h = np.array([
    [isqrt2, isqrt2],
    [isqrt2, -isqrt2],
])

cx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
])

sw = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
])

unitary_gates = {"x": x, "z": z, "h": h, "cx": cx, "sw": sw}

#### Parametric gates

$ U_3(\theta, \phi, \lambda) = \begin{pmatrix}
    \cos\frac{\theta}{2} & -e^{i\lambda}\sin\frac{\theta}{2}  \\
    e^{i\phi}\sin\frac{\theta}{2} & e^{i\phi + i\lambda}\cos\frac{\theta}{2} 
  \end{pmatrix}$

In [8]:

def param_gate(euler_theta, euler_phi, euler_lambda):
    
    u3 = np.array([
        [np.cos(euler_theta/2), -np.e**(euler_lambda*1j) * np.sin(euler_theta/2)],
        [np.e**(euler_phi*1j) * np.sin(euler_theta/2), np.e**(euler_phi*1j + euler_lambda*1j) * np.cos(euler_theta/2)]
    ])
    
    return u3

### Computational basis vector states

In [3]:
# computational basis
basis_states = [ [1, 0], [0, 1] ]

# classical bit states as strings
cbits = ["0", "1"]

### Quantum Circuit Simulator class with methods required for simulation

In [24]:
class QCSimulator:
    def __init__(self, qubits_num, shots, qcirc):
        """
        Takes a quantum circuit program, qcirc. Creates initial state vector
        of qubits_num qubits.
        """
        # initializer
        self.qubits_num = qubits_num
        self.shots_num = shots
        
        # create initial state vector of qubits_num qubits [1, 0, ... ]
        self.init_state = self.get_ground_state()
        print("Initial state = ", self.init_state, "\n")
        
        # number of gate operations to compute
        operations_num = len(qcirc)
        
        # circuit handler, read circuit, calculate matrix operator
        # multiply the state with the operator and return final state.
        final_state = self.circuit_handler(operations_num, qcirc)
        print("Final State vector = %s\n"%(final_state))
        
        # make measurements, get counts
        counts = self.get_counts(final_state)
        # print("\nMeasurement results:\n%s\n"%(counts))
        self.print_counts(counts)
        
    
    def get_ground_state(self):
        """Initialize a quantum state.
        Returns a vector of size 2**qubits_num with all zeroes except
        the first element which is 1."""
        ground_state_base = np.array([1, 0])
        ground_state_qnum = ground_state_base
        for i in range(self.qubits_num - 1):
            ground_state_qnum = np.kron(ground_state_base, ground_state_qnum)
            
        return ground_state_qnum
    
    def print_counts(self, counts):
        """Print out nicely looking count results"""
        print("\nMeasurement results:\n{")
        for i in counts.keys():
            print('\t"{}":\t{},'.format(str(i), counts[i]))
        print("}\n")
    
    def get_operator(self, gate_unitary, qubits_target):
        """Returns a unitary operator of size 2**n x 2**n for a given
        quantum gate and target number of qubits.
        Resizing the gate's martix to the dimension of the state vector"""
        # if matrix shape ==  state shape, return matrix and exit
        if np.shape(gate_unitary) == (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary
        elif np.shape(gate_unitary) > (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary
        
        # define 2x2 identity matrix
        I = np.identity(2)
        
        # a list of the order of kronecker product operations
        kron_order = []
        # number of iterations for adding gates into list
        ops = 1 + ( (2**self.qubits_num)/2 / np.shape(gate_unitary)[0])
        for k in range(int(ops)):
            if k == qubits_target:
                kron_order.append(gate_unitary)
            else:
                kron_order.append(I)
                
        gate_unitary_new = kron_order[0]
                
        for i in range(self.qubits_num - 1):
            # check if operator size/shape is equal to 2**n x 2**n, where n = qubits_num
            if np.shape(gate_unitary_new) == (2**self.qubits_num, 2**self.qubits_num):
                return gate_unitary_new
            
            print("\n{} (x) {}\n".format(gate_unitary_new, kron_order[i+1]))
            gate_unitary_new = np.kron(gate_unitary_new, kron_order[i+1])
            
        
        return gate_unitary_new
    
    
    def circuit_handler(self, operations_num, qcirc):
        """Handles quantum circuit input, managing operations with gates, operators
        and state vectors. Returns final state vector."""
        final_state = self.init_state
        
        for i in range(operations_num):
            gate_name = qcirc[i]["gate"]
            params = qcirc[i]["params"] if "params" in qcirc[i].keys() else {}
            target = qcirc[i]["target"]
            if len(target) == 1:
                target = target[0]
            elif len(target) == 2:
                target = target[1]
            print("Target qubit index: ", target)
            
            if gate_name in unitary_gates.keys():
                gate = unitary_gates[gate_name]
            elif gate_name not in unitary_gates.keys():
                gate = param_gate(params["theta"], params["phi"], params["lambda"])
            else:
                print("\n[-] Error! Gate not yet defined\n")
                break
            
            # resize gates, find matrix operator if necessary
            gate = self.get_operator(gate, target)
            print("%s-gate\n\b%s"%(gate_name.upper(), gate))
            
            # matrix vector multiplicaion operation
            try:
                final_state = np.dot(gate, final_state)
            except ValueError:
                print("\n[-] Error: Unitary gate shape: {} not aligned with state vector shape: {}\n".format(np.shape(gate), np.shape(final_state)))
                break
                
            print("Final state = %s\n"%(final_state))
            
        return final_state      
    
    
    def get_counts(self, state_vector):
        """Execute measurements in a loop shots_num times and returns a dictionary object
        with statistics in the form:
        {
            element_index: number_of_occurrences,
            ...
        }
        only from elements which occurred - returned from measrements."""
        # find list of all possible quantum states
        qstates = basis_states
        cstates = cbits
        for i in range(self.qubits_num - 1):
            qstates = [ list(np.kron(j, k)) for j in basis_states for k in qstates ]
            cstates = [ j + k for j in cbits for k in cstates ]
        
        # fill in probabilities of each quantum state
        prob_states = []
        for i in range( len(qstates) ):
            p = np.dot( qstates[i], state_vector )
            pp = np.dot(p, np.conj(p)) 
            prob_states.append( pp.real )
            
        print("Weighted probabilitites:\n{}".format(prob_states))
        
        # loop measurement using weighted random technique
        # dictionary to hold statistics of states and their counts
        counts = {}
        counter = 0
        for i in range( self.shots_num ):
            collapsed_state = choice( cstates, 1, p=prob_states )
            if collapsed_state[0] in counts:
                counts[collapsed_state[0]] += 1
            elif not collapsed_state[0] in counts:
                counts[collapsed_state[0]] = 1
                
        return counts

    

N = 2   # number of qubits for the state vector; 2-qubits state vector
shots = 1024    # number of executions

# quantum circuit/program
circ = [
    #{"gate": "x", "target": [1]},
    #{"gate": "x", "target": [0]},
    #{"gate": "z", "target": [0]},
    #{"gate": "h", "target": [2]},
    {"gate": "u3", "params": {"theta": 3.1415, "phi": 1.5708, "lambda": -3.1415}, "target": [0]},
    {"gate": "h", "target": [0]},
    {"gate": "cx", "target": [0, 1]},
]

QCSimulator(N, shots, circ)

Initial state =  [1 0 0 0] 

Target qubit index:  0

[[ 4.63267949e-05+0.00000000e+00j  9.99999995e-01+9.26535896e-05j]
 [-3.67320510e-06+9.99999999e-01j  4.46251166e-09-4.63267947e-05j]] (x) [[1. 0.]
 [0. 1.]]

U3-gate
[[ 4.63267949e-05+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   9.99999995e-01+9.26535896e-05j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  4.63267949e-05+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  9.99999995e-01+9.26535896e-05j]
 [-3.67320510e-06+9.99999999e-01j -0.00000000e+00+0.00000000e+00j
   4.46251166e-09-4.63267947e-05j  0.00000000e+00+0.00000000e+00j]
 [-0.00000000e+00+0.00000000e+00j -3.67320510e-06+9.99999999e-01j
   0.00000000e+00+0.00000000e+00j  4.46251166e-09-4.63267947e-05j]]
Final state = [ 4.63267949e-05+0.j  0.00000000e+00+0.j -3.67320510e-06+1.j
  0.00000000e+00+0.j]

Target qubit index:  0

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]] (x) [[1. 0.]
 [0. 1.]]

H-gate
[[ 0.70710678  0.          0.70710678